In [32]:
import pickle

df = pickle.load(open("data/parsed_docs_with_toks.pkl", "rb"))

In [33]:
df

,docno,text,tokens
0,1050324_business_story_4530765.utf8,The Telegraph - Calcutta : Business Tighter ...,"[telegraph, calcutta, business, tighter, norms..."
1,1050907_business_story_5207471.utf8,The Telegraph - Calcutta : Business Global S...,"[telegraph, calcutta, business, global, steel,..."
2,1050811_business_story_5100703.utf8,The Telegraph - Calcutta : Business RBI eagl...,"[telegraph, calcutta, business, rbi, eagles, o..."
3,1050610_business_story_4849980.utf8,The Telegraph - Calcutta : Business Posco de...,"[telegraph, calcutta, business, posco, deal, m..."
4,1051226_business_story_5645448.utf8,The Telegraph - Calcutta : Business LOOSE CH...,"[telegraph, calcutta, business, loose, change,..."
...,...,...,...
392534,en.3.403.491.2007.11.7,"SINGAPORE, Nov 7 (bdnews24.com/Reuters) - Oil ...","[singapore, nov, 7, bdnews24comreuters, oil, p..."
392535,en.3.403.360.2007.11.4,"CHICAGO, Nov 4 (bdnews24.com/Reuters) - An eff...","[chicago, nov, 4, bdnews24comreuters, effort, ..."
392536,en.3.403.430.2007.11.5,"PARIS, Nov 5 (bdnews24.com/Reuters) - Airbus p...","[paris, nov, 5, bdnews24comreuters, airbus, pa..."
392537,en.3.403.407.2007.11.6,"Pabna, Nov 5 (bdnews24.com) Awami League lead...","[pabna, nov, 5, bdnews24com, awami, league, le..."


In [4]:
from rank_bm25 import BM25Okapi

idx = BM25Okapi(df["tokens"].tolist())

In [5]:
import pickle 

pickle.dump(idx, open("saved_indices/bm250kapi_idx.pkl", "wb"))


In [6]:
import spacy 
from spacy.lang.en.stop_words import STOP_WORDS
import re
import swifter
import pandas as pd  
import pickle

In [28]:
import torch
import numpy as np 

num_results = 100

def get_rank25_results(query: str, idx, num_results:int = 25):
    scores = idx.get_scores(query)
    sorted_score_idx = np.argsort(scores)[::-1]

    results = {}
    for i in sorted_score_idx[:num_results]:
        docno = df.iloc[i]["docno"]
        score = scores[i].item()
        results[docno] = score
    return results    

In [7]:
from utils import read_qrel_from_file, evaluate_run

In [8]:
qrel = read_qrel_from_file("qrels/2020/cair2020_qrel.txt")

In [23]:
from bs4 import BeautifulSoup

fp = open("qrels/2020/topics_test.txt")
soup = BeautifulSoup(fp, 'xml')
soup.find_all("top")

extracted_topics = []
for topic in soup.find_all("top"):
    number = str(topic.num.text)
    title = topic.title.text.strip()
    narr = topic.narr.text.strip()
    extracted_topics.append({"number": number, "title": title, "narrative": narr})

df = pd.DataFrame(extracted_topics)
df

,number,title,narrative
0,1,Assassination of Osama-bin-laden,Relevant document should contain information a...
1,2,Accused Ajmal Kasab,Information on Kasab's confession about 26/11 ...
2,3,Maharashtra CM ashok chavan resignation,Documents about Ashok Chavan's deliberate endi...
3,4,Accused Sanjay Dutt,Relevant document would highlight that actor S...
4,5,Abu Salem accused,Relevants documents will contain evidences on ...
5,6,Babri Masjid demolition case against Advani,A relevant document must include information a...
6,7,CBI searches Dawood Ibrahim,A relevant document must provide information r...
7,8,Court blocks facebook in pakistan,Relevant documents should contain information ...
8,9,Jaswant Singh BJP sacking,Relevant documents should contain information ...
9,10,Kasab's nationality confirmed,A relevant document must provide information o...


In [25]:
def extract_topics_from_file(file: str) -> dict:
    soup = BeautifulSoup(open(file, "r"), 'xml')

    extracted_topics = []
    for topic in soup.find_all("top"):
        number = str(topic.num.text)
        title = topic.title.text.strip()
        narr = topic.narr.text.strip()
        extracted_topics.append({"number": number, "title": title, "narrative": narr})
    
    
    return pd.DataFrame(extracted_topics)

topics = extract_topics_from_file("qrels/2020/topics_test.txt")

topics

,number,title,narrative
0,1,Assassination of Osama-bin-laden,Relevant document should contain information a...
1,2,Accused Ajmal Kasab,Information on Kasab's confession about 26/11 ...
2,3,Maharashtra CM ashok chavan resignation,Documents about Ashok Chavan's deliberate endi...
3,4,Accused Sanjay Dutt,Relevant document would highlight that actor S...
4,5,Abu Salem accused,Relevants documents will contain evidences on ...
5,6,Babri Masjid demolition case against Advani,A relevant document must include information a...
6,7,CBI searches Dawood Ibrahim,A relevant document must provide information r...
7,8,Court blocks facebook in pakistan,Relevant documents should contain information ...
8,9,Jaswant Singh BJP sacking,Relevant documents should contain information ...
9,10,Kasab's nationality confirmed,A relevant document must provide information o...


In [29]:
import pytrec_eval
metrics: set = {'map', 'ndcg', 'recall'}

evaluator = pytrec_eval.RelevanceEvaluator(qrel, metrics)
evaluator.evaluate(run)


NameError: name 'run' is not defined

In [26]:
import spacy

nlp = spacy.load("en_core_web_sm", disable=["tok2vec", "ner"])

def preprocess_query(text):
    clean_text = re.sub(r'[^\w\s]', '', text.lower())
    
    toks = []
    for tok in nlp(clean_text):
        if tok.text not in STOP_WORDS and tok.text.strip() != "":
            toks.append(tok.lemma_)
    return toks


In [34]:
import tqdm.notebook as tqdm
import numpy as np
title_run = {}
narr_run = {}

for topic in tqdm.tqdm(extracted_topics):
    title_toks = preprocess_query(topic["title"])
    narr_toks = preprocess_query(topic["narrative"])

    title_results = get_rank25_results(title_toks, idx, 50)
    narr_results = get_rank25_results(narr_toks, idx, 50)
    
    title_run[topic["number"]] = title_results
    narr_run[topic["number"]] = narr_results
print("finished")


finished


In [35]:
import pytrec_eval
import numpy as np 
import json 

metrics: set = {'map', 'ndcg', 'P_5'}

evaluator = pytrec_eval.RelevanceEvaluator(qrel, metrics)
run_results = evaluator.evaluate(title_run)

map_scores = [v["map"] for k,v in run_results.items()]
p_scores  = [v["P_5"] for k,v in run_results.items()]

display(run_results)

print("Aggregate results")
print("Average MAP: ", np.mean(map_scores))
print("Average P_5: ", np.mean(p_scores))


{'1': {'map': 0.0, 'P_5': 0.0, 'ndcg': 0.0},
 '2': {'map': 0.056925343751603366, 'P_5': 0.4, 'ndcg': 0.21217850514857134},
 '3': {'map': 0.22794518761345814, 'P_5': 0.2, 'ndcg': 0.5222640046935143},
 '4': {'map': 0.20934800847205273, 'P_5': 0.4, 'ndcg': 0.4284309176998409},
 '5': {'map': 0.40266188901243055, 'P_5': 0.8, 'ndcg': 0.5785227009091111},
 '6': {'map': 0.19304223216365549, 'P_5': 0.4, 'ndcg': 0.4385255861592022},
 '7': {'map': 0.2886084338895777, 'P_5': 0.8, 'ndcg': 0.5275960595929252},
 '8': {'map': 0.6363636363636364, 'P_5': 1.0, 'ndcg': 0.8133975486119581},
 '9': {'map': 0.16931596118777784, 'P_5': 0.6, 'ndcg': 0.37475773324028167},
 '10': {'map': 0.1559469696969697, 'P_5': 0.4, 'ndcg': 0.3779016492452397},
 '11': {'map': 0.11291291023013414, 'P_5': 0.4, 'ndcg': 0.2719987846025049},
 '12': {'map': 0.19806517661348838, 'P_5': 0.4, 'ndcg': 0.4530618556493753},
 '13': {'map': 0.3148126258703182, 'P_5': 1.0, 'ndcg': 0.5245829186167061},
 '14': {'map': 0.6165193866105699, 'P_5'

Aggregate results
Average MAP:  0.2648341415668153
Average P_5:  0.54


In [113]:
import pytrec_eval
import numpy as np 
import json 

metrics: set = {'map', 'ndcg', 'P_5'}

evaluator = pytrec_eval.RelevanceEvaluator(qrel, metrics)
run_results = evaluator.evaluate(title_run)

map_scores = [v["map"] for k,v in run_results.items()]
p_scores  = [v["P_5"] for k,v in run_results.items()]

display(run_results)

print("Aggregate results")
print("Average MAP: ", np.mean(map_scores))
print("Average P_5: ", np.mean(p_scores))

# with open("results/bm250kap-baseline/title-results.txt", "w") as f:
#     f.write(f"Average MAP: {np.mean(map_scores)}\n")
#     f.write(f"Average P_5: {np.mean(p_scores)}\n")
#     f.write(f"Individual run results: \n")
#     f.write(json.dumps(run_results, indent=4))

# with open("results/bm250kap-baseline/title-run.txt", "w") as f:
#     f.write(json.dumps(title_run, indent=4))

{'1': {'map': 0.0857992729838509, 'P_5': 0.4, 'ndcg': 0.24541403340349474},
 '2': {'map': 0.09291773146996271, 'P_5': 0.4, 'ndcg': 0.26489068160802665},
 '3': {'map': 0.21966240903604048, 'P_5': 0.2, 'ndcg': 0.5163460600226231},
 '4': {'map': 0.06638540431013955, 'P_5': 0.4, 'ndcg': 0.24807734635625595},
 '5': {'map': 0.4161339977285736, 'P_5': 1.0, 'ndcg': 0.5852790666091732},
 '6': {'map': 0.20315442597801556, 'P_5': 0.2, 'ndcg': 0.451153225278768},
 '7': {'map': 0.2505158547737754, 'P_5': 0.8, 'ndcg': 0.5015713626019195},
 '8': {'map': 0.45906334656334646, 'P_5': 0.4, 'ndcg': 0.7473829770969648},
 '9': {'map': 0.18853064242175385, 'P_5': 0.2, 'ndcg': 0.4168872426997385},
 '10': {'map': 0.08608322197031876, 'P_5': 0.4, 'ndcg': 0.2740905118763117},
 '11': {'map': 0.07576138370159298, 'P_5': 0.4, 'ndcg': 0.24505669724589946},
 '12': {'map': 0.40259024538508315, 'P_5': 0.6, 'ndcg': 0.6454313209816381},
 '13': {'map': 0.2676652326251622, 'P_5': 0.4, 'ndcg': 0.48800951596961595},
 '14': {

Aggregate results
Average MAP:  0.2561636552683072
Average P_5:  0.4999999999999999


In [36]:
run_results = evaluator.evaluate(narr_run)

map_scores = [v["map"] for k,v in run_results.items()]
p_scores  = [v["P_5"] for k,v in run_results.items()]

display(run_results)

print("Aggregate results")
print("Average MAP: ", np.mean(map_scores))
print("Average P_5: ", np.mean(p_scores))

{'1': {'map': 0.04402549433441882, 'P_5': 0.4, 'ndcg': 0.17227327297591483},
 '2': {'map': 0.1868069134882157, 'P_5': 0.6, 'ndcg': 0.3866051581702334},
 '3': {'map': 0.7272447414197384, 'P_5': 1.0, 'ndcg': 0.9088433466673372},
 '4': {'map': 0.31381425763566484, 'P_5': 1.0, 'ndcg': 0.5202545637719598},
 '5': {'map': 0.28162893887628765, 'P_5': 1.0, 'ndcg': 0.46215507858247157},
 '6': {'map': 0.24250418514395317, 'P_5': 0.6, 'ndcg': 0.5136410432069678},
 '7': {'map': 0.268405442651054, 'P_5': 0.8, 'ndcg': 0.49828958478148627},
 '8': {'map': 0.0047619047619047615, 'P_5': 0.0, 'ndcg': 0.04935421935348238},
 '9': {'map': 0.4642077787980886, 'P_5': 0.4, 'ndcg': 0.6835174615774817},
 '10': {'map': 0.44197706381403146, 'P_5': 1.0, 'ndcg': 0.6589136441994532},
 '11': {'map': 0.07705919166748026, 'P_5': 0.6, 'ndcg': 0.24716036726857735},
 '12': {'map': 0.3827854210316024, 'P_5': 0.4, 'ndcg': 0.6323273456684958},
 '13': {'map': 0.27428235399992373, 'P_5': 0.2, 'ndcg': 0.4639665499009099},
 '14': 

Aggregate results
Average MAP:  0.32489416116275116
Average P_5:  0.62


In [114]:
run_results = evaluator.evaluate(narr_run)

map_scores = [v["map"] for k,v in run_results.items()]
p_scores  = [v["P_5"] for k,v in run_results.items()]

display(run_results)

print("Aggregate results")
print("Average MAP: ", np.mean(map_scores))
print("Average P_5: ", np.mean(p_scores))

with open("results/bm250kap-baseline/narrative-results.txt", "w") as f:
    f.write(f"Average MAP: {np.mean(map_scores)}\n")
    f.write(f"Average P_5: {np.mean(p_scores)}\n")
    f.write(f"Individual run results: \n")
    f.write(json.dumps(run_results, indent=4))

with open("results/bm250kap-baseline/narrative-run.txt", "w") as f:
    f.write(json.dumps(title_run, indent=4))

{'1': {'map': 0.24554547962813247, 'P_5': 1.0, 'ndcg': 0.43195244767143376},
 '2': {'map': 0.09316836010136732, 'P_5': 0.2, 'ndcg': 0.26286249092777214},
 '3': {'map': 0.6781836591938896, 'P_5': 1.0, 'ndcg': 0.8419504708828816},
 '4': {'map': 0.20881728355238408, 'P_5': 0.6, 'ndcg': 0.39788740206231976},
 '5': {'map': 0.30906608872874, 'P_5': 1.0, 'ndcg': 0.49363714531501374},
 '6': {'map': 0.21590003848037764, 'P_5': 0.8, 'ndcg': 0.46805865253277174},
 '7': {'map': 0.22486683527503215, 'P_5': 0.8, 'ndcg': 0.4615856394639282},
 '8': {'map': 0.025, 'P_5': 0.2, 'ndcg': 0.09478836436955078},
 '9': {'map': 0.49343012855076335, 'P_5': 0.6, 'ndcg': 0.7221873256895347},
 '10': {'map': 0.46085418124891814, 'P_5': 1.0, 'ndcg': 0.6665286161517171},
 '11': {'map': 0.03805272719690148, 'P_5': 0.0, 'ndcg': 0.15358268625049804},
 '12': {'map': 0.48891263513879235, 'P_5': 0.8, 'ndcg': 0.7262392552464187},
 '13': {'map': 0.24857027763549955, 'P_5': 0.4, 'ndcg': 0.45001064894167886},
 '14': {'map': 0.2

Aggregate results
Average MAP:  0.3285357429896214
Average P_5:  0.6399999999999999
